# 02 Batch profiles / バッチ処理

Process a folder of GLBs (synthetic in this demo), build individual profiles, and overlay them for comparisons.

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tire_profiler.align import align_points
from tire_profiler.cylinder import fit_cylinder
from tire_profiler.rimline import fit_rimline
from tire_profiler.slice_profile import cylindrical_features, slice_band, compute_profile

In [ ]:
def synthetic_tire_points(seed: int, offset: float = 0.0, n: int = 100_000):
    rng = np.random.default_rng(seed)
    x = rng.uniform(-0.12, 0.12, n) + offset
    theta = rng.uniform(-np.pi, np.pi, n)
    radius = 0.34 + 0.003 * np.sin(4 * theta + offset)
    y = radius * np.sin(theta)
    z = radius * np.cos(theta)
    rim_mask = np.abs(x - offset) > 0.09
    z[rim_mask] += 0.004 * np.exp(-((theta[rim_mask]) ** 2) / 0.08)
    pts = np.stack([x, y, z], axis=1) + rng.normal(scale=0.0007, size=(n, 3))
    return pts

In [ ]:
def run_profile(seed: int) -> pd.DataFrame:
    pts = synthetic_tire_points(seed, offset=0.0)
    model = fit_cylinder(pts, threshold=0.0025)
    aligned, *_ = align_points(pts, model.point_on_axis, model.axis_direction)
    features = cylindrical_features(aligned, model.radius)
    mask = slice_band(aligned, features=features, tape_width=0.02, outer_band=0.05)
    rim_mask = (np.abs(aligned[:, 0]) > 0.11) & (np.abs(features['arc']) < 0.03)
    rim_points = aligned[rim_mask][:20]
    arc = np.arctan2(rim_points[:, 1], rim_points[:, 2]) * model.radius
    rimline = fit_rimline(rim_points, arc)
    result = compute_profile(aligned, features=features, mask=mask, rimline=rimline, nbins=80)
    return result.profile

In [ ]:
profiles = []
for idx, seed in enumerate([0, 1, 2]):
    df = run_profile(seed)
    df = df.assign(tire=f"Tire {idx+1}")
    profiles.append(df)
combined = pd.concat(profiles, ignore_index=True)
combined.head()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
for tire, grp in combined.groupby('tire'):
    ax.plot(grp['x_center'], grp['z_mean'], label=tire)
ax.set_xlabel('X (axial)')
ax.set_ylabel("Z' (rim zero)")
ax.legend()
ax.set_title('Overlay of tire profiles')
ax.grid(True, alpha=0.3)
plt.show()

In [ ]:
stacked = combined.groupby(['tire', 'x_center']).agg(z_mean=('z_mean', 'mean')).reset_index()
agg = stacked.groupby('x_center').agg(mean=('z_mean', 'mean'), std=('z_mean', 'std'))
agg.head()